In [31]:
%matplotlib inline

import glob
import pandas as pd
import yaml
import numpy as np
import cv2
import time
from skimage import exposure
from skimage import feature
import matplotlib.pyplot as plt

In [33]:
show_image = False

def show_hog_img(img, hog_img):
    for i in range(4):
        hog_img[i] = exposure.rescale_intensity(hog_img[i], out_range=(0, 255))
        hog_img[i] = hog_img[i].astype("uint8")

    img_concat = np.concatenate((img[0], img[1],img[2],img[3]), axis = 1)
    img_hog_concat = np.concatenate((hog_img[0], hog_img[1],hog_img[2],hog_img[3]), axis = 1)
    
    img_total = np.concatenate((img_concat, img_hog_concat), axis = 0)
    
    fig, ax = plt.subplots(nrows=1, ncols=4)
    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.hist(hog_img[i])
    plt.show()

    cv2.imshow("HOG Image", img_total)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [18]:
ds = pd.read_csv('training.csv', header=0)

In [19]:
ds.head()

,class,person,depfile_path,depxs,depys,depxe,depye,ir1file_path,ir1xs,ir1ys,...,ir2file_path,ir2xs,ir2ys,ir2xe,ir2ye,rgbfile_path,rgbxs,rgbys,rgbxe,rgbye
0,1,p002,.\p002\02_1_depth_1.bmp,35,120,215,300,.\p002\02_1_ir1_1.bmp,35,120,...,.\p002\02_1_ir2_1.bmp,35,80,215,260,.\p002\02_1_rgb_1.bmp,30,90,210,270
1,1,p002,.\p002\02_1_depth_2.bmp,35,120,215,300,.\p002\02_1_ir1_2.bmp,35,120,...,.\p002\02_1_ir2_2.bmp,35,80,215,260,.\p002\02_1_rgb_2.bmp,30,90,210,270
2,1,p002,.\p002\02_1_depth_3.bmp,35,120,215,300,.\p002\02_1_ir1_3.bmp,35,120,...,.\p002\02_1_ir2_3.bmp,35,80,215,260,.\p002\02_1_rgb_3.bmp,30,90,210,270
3,1,p002,.\p002\02_1_depth_4.bmp,35,120,215,300,.\p002\02_1_ir1_4.bmp,35,120,...,.\p002\02_1_ir2_4.bmp,35,80,215,260,.\p002\02_1_rgb_4.bmp,30,90,210,270
4,1,p002,.\p002\02_1_depth_5.bmp,35,120,215,300,.\p002\02_1_ir1_5.bmp,35,120,...,.\p002\02_1_ir2_5.bmp,35,80,215,260,.\p002\02_1_rgb_5.bmp,30,90,210,270


In [48]:
data_hog = []

In [49]:
def calculate_hog(row):
    tic = time.clock()
#for index, row in ds.iterrows():
    print(ds.shape, len(data_hog), row['depfile_path'])
    # improvement agrupar
    dep = cv2.imread(row['depfile_path'],0)
    dep = dep[row['depxs']:row['depxe'],row['depys']:row['depye']]
    (H, dep_hog) = feature.hog(dep, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, visualise=True)
    
    ir1 = cv2.imread(row['ir1file_path'],0)
    ir1 = ir1[row['ir1xs']:row['ir1xe'],row['ir1ys']:row['ir1ye']]
    (H, ir1_hog) = feature.hog(ir1, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, visualise=True)

    ir2 = cv2.imread(row['ir2file_path'],0)
    ir2 = ir2[row['ir2xs']:row['ir2xe'],row['ir2ys']:row['ir2ye']]
    (H, ir2_hog) = feature.hog(ir2, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, visualise=True)

    rgb = cv2.imread(row['rgbfile_path'],0)
    rgb = rgb[row['rgbxs']:row['rgbxe'],row['rgbys']:row['rgbye']]
    (H, rgb_hog) = feature.hog(rgb, orientations=9, pixels_per_cell=(4, 4),cells_per_block=(2, 2), transform_sqrt=True, visualise=True)

    data_hog.append([row['class'], row['person'], dep_hog.flatten(), ir1_hog.flatten(), ir2_hog.flatten(), rgb_hog.flatten()])
    
    if show_image == True:
        show_hog_img([dep,ir1,ir2,rgb], [dep_hog, ir1_hog, ir2_hog, rgb_hog])
    print(time.clock() - tic)        

511.85454396767443


In [50]:
ds.apply(calculate_hog, axis = 1)

(1235, 22) 0 .\p002\02_1_depth_2.bmp
(1235, 22) 1 .\p002\02_1_depth_3.bmp
(1235, 22) 2 .\p002\02_1_depth_4.bmp
(1235, 22) 3 .\p002\02_1_depth_5.bmp
(1235, 22) 4 .\p002\02_1_depth_6.bmp
(1235, 22) 5 .\p002\02_1_depth_7.bmp
(1235, 22) 6 .\p002\02_1_depth_8.bmp
(1235, 22) 7 .\p002\02_1_depth_9.bmp
(1235, 22) 8 .\p002\02_1_depth_10.bmp


1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

In [51]:
data_hog

[[1, 'p002', array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          1.99609375,  0.        ]), array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.43530273,  0.        ]), array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.07275391,  0.        ]), array([ 0.,  0.,  0., ...,  0.,  0.,  0.])],
 [1, 'p002', array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          1.99609375,  0.        ]), array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.41455078,  0.        ]), array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.12040012,  0.        ]), array([ 0.,  0.,  0., ...,  0.,  0.,  0.])],
 [1, 'p002', array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.99804688,  0.        ]), array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.40600586,  0.        ]), array([ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.0781

In [52]:
# save file
df = pd.DataFrame(data_hog, columns = ['class','person', 'dep_hog', 'ir1_hog', 'ir2_hog', 'rgb_hog'])
filename = 'training_hog.csv'
df.to_csv(filename, index=False, encoding='utf-8')

In [53]:
df.head()

,class,person,dep_hog,ir1_hog,ir2_hog,rgb_hog
0,1,p002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,p002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,p002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,p002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,p002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
